## 1. Environment Setup

In [25]:
!pip install -q torch torchvision
!pip install -q ultralytics
!pip install -q huggingface_hub
!pip install -q opencv-python matplotlib pandas seaborn
!pip install datasets huggingface-hub

In [2]:
import os
import torch
import torchvision
import numpy as np
import cv2
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download
from pathlib import Path
import shutil
import random
import time
from tqdm.auto import tqdm
import yaml

In [3]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Running on CPU")

CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May  9 09:20:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             45W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
# Create directories
os.makedirs("data", exist_ok=True)
os.makedirs("weights", exist_ok=True)
os.makedirs("results", exist_ok=True)

## 2. Data Acquisition and Preprocessing

In [43]:
def download_pdt_dataset():
    """
    Download a sample of the PDT dataset and place it in the correct location for Ultralytics.
    """
    # The correct base directory for Ultralytics datasets
    base_dir = "/content/datasets"
    dataset_dir = os.path.join(base_dir, "PDT")

    # Create directories
    for split in ["train", "val"]:
        os.makedirs(os.path.join(dataset_dir, "images", split), exist_ok=True)
        os.makedirs(os.path.join(dataset_dir, "labels", split), exist_ok=True)

    print("Creating placeholder data for demonstration...")

    # Create placeholder images and labels
    for split in ["train", "val"]:
        num_images = 20 if split == "train" else 5
        for i in range(num_images):
            # Create a black image with a white rectangle as an "unhealthy" area
            img = np.zeros((640, 640, 3), dtype=np.uint8)
            x1, y1 = random.randint(100, 400), random.randint(100, 400)
            x2, y2 = x1 + random.randint(50, 200), y1 + random.randint(50, 200)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), -1)

            # Save image
            img_path = os.path.join(dataset_dir, "images", split, f"placeholder_{i}.jpg")
            cv2.imwrite(img_path, img)

            # Save label (YOLO format: class x_center y_center width height)
            label_path = os.path.join(dataset_dir, "labels", split, f"placeholder_{i}.txt")

            # Calculate YOLO format coordinates
            img_width, img_height = 640, 640
            x_center = (x1 + x2) / 2 / img_width
            y_center = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            with open(label_path, "w") as f:
                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    print(f"Created placeholder data in {dataset_dir}")
    return dataset_dir

In [44]:
def prepare_data_config(dataset_path):
    """Create YAML config file for YOLO training"""
    data_yaml = {
        'path': dataset_path,  # Use the absolute path
        'train': 'images/train',  # Relative to path
        'val': 'images/val',      # Relative to path
        'names': {0: 'unhealthy'},
        'nc': 1  # number of classes
    }

    config_path = os.path.join(dataset_path, "data.yaml")
    with open(config_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f"Created data configuration at {config_path}")
    return config_path

## 3. YOLO-DP Model Architecture

In [45]:
class GhostConv(torch.nn.Module):
    """Ghost Convolution from GhostNet"""
    def __init__(self, c1, c2, k=1, s=1, g=1, act=True):
        super(GhostConv, self).__init__()
        c_ = c2 // 2  # hidden channels
        self.cv1 = torch.nn.Conv2d(c1, c_, k, s, k//2, groups=g, bias=False)
        self.cv2 = torch.nn.Conv2d(c_, c_, 3, 1, 1, groups=c_, bias=False)
        self.bn = torch.nn.BatchNorm2d(c2)
        self.act = torch.nn.SiLU() if act else torch.nn.Identity()

    def forward(self, x):
        y = self.cv1(x)
        return self.act(self.bn(torch.cat((y, self.cv2(y)), 1)))

In [46]:
class AdaptiveLargeScaleSelectiveKernel(torch.nn.Module):
    """
    Adaptive Large Scale Selective Kernel as described in YOLO-DP paper
    For capturing the location information of dense, small target pest-infested trees
    """
    def __init__(self, c1, c2):
        super(AdaptiveLargeScaleSelectiveKernel, self).__init__()
        self.conv1 = torch.nn.Conv2d(c1, c2, kernel_size=5, padding=2)  # Shallow range
        self.conv2 = torch.nn.Conv2d(c2, c2, kernel_size=7, padding=9)  # Deep range

        # GhostConv for feature selection
        self.ghost1 = GhostConv(c2, c2, k=1)
        self.ghost2 = GhostConv(c2, c2, k=1)

        # Spatial attention
        self.spatial_attn = torch.nn.Sequential(
            torch.nn.Conv2d(2, 1, kernel_size=7, padding=3),
            torch.nn.Sigmoid()
        )

        self.final_conv = torch.nn.Conv2d(c2, c2, kernel_size=1)

    def forward(self, x):
        # Extract shallow and deep range info
        m1 = self.conv1(x)
        m2 = self.conv2(m1)

        # Feature selection
        m3 = self.ghost1(m1)
        m4 = self.ghost2(m2)

        # Spatial attention
        avg_pool = torch.mean(torch.cat([m3, m4], dim=1), dim=1, keepdim=True)
        max_pool, _ = torch.max(torch.cat([m3, m4], dim=1), dim=1, keepdim=True)
        spatial_attn = self.spatial_attn(torch.cat([avg_pool, max_pool], dim=1))

        # Apply attention and combine
        m12 = m3 * spatial_attn
        m13 = m4 * spatial_attn
        output = self.final_conv(m12 + m13)

        return output

In [47]:
# This is a simplified implementation of YOLO-DP
# In practice, i would modify the YOLOv5 code directly
# or use the Ultralytics YOLO framework with custom modules

def create_yolo_dp_model(pretrained=False):
    """
    Create a YOLO-DP model.
    For simplicity, we'll use Ultralytics YOLOv5 as the base and
    describe how you would customize it to implement YOLO-DP
    """
    # In a real implementation, you would modify the YOLOv5 code directly
    # Here we're using YOLOv5s as a base and explaining the modifications

    print("Creating YOLO-DP model based on YOLOv5s...")

    # For actual implementation:
    # 1. Clone the YOLOv5 repository
    # 2. Modify models/common.py to add GhostConv and AdaptiveLargeScaleSelectiveKernel
    # 3. Modify models/yolo.py to use these modules in the backbone and neck
    # 4. Implement decoupled detection heads

    # For this example, we'll use YOLOv5s from Ultralytics
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=pretrained)

    print("YOLO-DP model created (simplified version)!")

    # Add a note about the actual implementation
    print("\nNote: This is a simplified version of YOLO-DP.")
    print("For a full implementation, you would need to modify the YOLOv5 architecture")
    print("by integrating the Adaptive Large Scale Selective Kernel and GhostConv modules")
    print("as described in the paper.")

    return model

In [48]:
# Create the model
model = create_yolo_dp_model(pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-9 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)

Fusing layers... 


Creating YOLO-DP model based on YOLOv5s...


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLO-DP model created (simplified version)!

Note: This is a simplified version of YOLO-DP.
For a full implementation, you would need to modify the YOLOv5 architecture
by integrating the Adaptive Large Scale Selective Kernel and GhostConv modules
as described in the paper.


## 4. Training Pipeline

In [49]:
def train_model(data_config, epochs=100, batch_size=16, img_size=640):
    """
    Train the YOLO-DP model.
    Using YOLOv5 training pipeline with custom configuration.
    """
    print(f"Starting training for {epochs} epochs...")

    # Use YOLOv5s as the base model (newer versions might allow yolov5su.pt)
    !yolo train \
        model=yolov5s.pt \
        data={data_config} \
        epochs={epochs} \
        batch={batch_size} \
        imgsz={img_size} \
        patience=100 \
        project=runs/train \
        name=YOLO-DP \
        exist_ok=True

    print("Training completed!")
    return "runs/train/YOLO-DP/weights/best.pt"

In [50]:
# Run the pipeline
dataset_path = download_pdt_dataset()
data_config = prepare_data_config(dataset_path)
best_weights = train_model(data_config, epochs=10)

Creating placeholder data for demonstration...
Created placeholder data in /content/datasets/PDT
Created data configuration at /content/datasets/PDT/data.yaml
Starting training for 10 epochs...
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/PDT/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipu

## 5. Evaluation

In [51]:
def evaluate_model(model_path, data_config, batch_size=16, img_size=640):
    """Evaluate the trained model."""
    print(f"Evaluating model from {model_path}...")

    # Using YOLOv5's val.py script for evaluation
    !yolo val \
        model={model_path} \
        data={data_config} \
        batch={batch_size} \
        imgsz={img_size} \
        project=runs/val \
        name=YOLO-DP-eval \
        exist_ok=True

    print("Evaluation completed!")

In [52]:
evaluate_model(best_weights, data_config)

Evaluating model from runs/train/YOLO-DP/weights/best.pt...
Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv5s summary (fused): 84 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 506.7±110.0 MB/s, size: 8.3 KB)
val: Scanning /content/datasets/PDT/labels/val.cache... 5 images, 0 backgrounds, 0 corrupt: 100% 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:00<00:00,  3.41it/s]
                   all          5          5      0.973          1      0.995      0.899
Speed: 1.2ms preprocess, 11.4ms inference, 0.0ms loss, 27.0ms postprocess per image
Results saved to runs/val/YOLO-DP-eval
💡 Learn more at https://docs.ultralytics.com/modes/val
Evaluation completed!


## 6. Visualization and Inference

In [53]:
def visualize_predictions(model_path, img_path, conf_threshold=0.25):
    """Visualize model predictions on a single image."""
    # Load model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    model.conf = conf_threshold

    # Make prediction
    results = model(img_path)

    # Visualize
    results.show()

    # Save results
    output_path = f"results/prediction_{Path(img_path).stem}.jpg"
    results.save(save_dir="results")
    print(f"Saved prediction to {output_path}")

    return results

In [55]:
def inference_on_folder(weights_path, folder_path, conf_threshold=0.25, save_dir="results"):
    """
    Run inference on all images in a specified folder.

    Args:
        weights_path: Path to the trained model weights
        folder_path: Path to the folder containing images to run inference on
        conf_threshold: Confidence threshold for detections (0-1)
        save_dir: Directory to save results

    Returns:
        output_path: Path to the saved results
    """
    # Check if weights exist
    if not os.path.exists(weights_path):
        print(f"Error: Weights file not found at {weights_path}")
        return None

    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder not found at {folder_path}")
        return None

    print(f"Running inference on folder: {folder_path}")
    print(f"Using model weights: {weights_path}")
    print(f"Confidence threshold: {conf_threshold}")

    # Create output directory
    os.makedirs(save_dir, exist_ok=True)

    # Run inference using YOLO command
    output_path = os.path.join(save_dir, "pdt_predictions")

    # Use Ultralytics YOLO for inference
    !yolo predict model={weights_path} source={folder_path} conf={conf_threshold} project={save_dir} name=pdt_predictions save=True

    print(f"Inference complete! Results saved to {output_path}")

    # Count the number of prediction images
    try:
        num_predictions = len(list(Path(output_path).glob("*.jpg")))
        print(f"Generated {num_predictions} prediction images")
    except Exception as e:
        print(f"Could not count prediction images: {e}")

    return output_path

In [57]:
# Path to your trained weights
best_weights = "runs/train/YOLO-DP/weights/best.pt"

# Path to folder with images for inference
test_folder = "path/to/test/folder"  # Replace with your actual test folder path

# Run inference
results_path = inference_on_folder(best_weights, test_folder, conf_threshold=0.25)

# Print results path
print(f"Inference results saved to: {results_path}")

Error: Folder not found at path/to/test/folder
Inference results saved to: None


## 7. Comparison with Other Models

In [ ]:
def compare_models():
    """
    Compare the performance of YOLO-DP with other models as shown in the paper.
    This is for reference; actual implementation would depend on the models.
    """
    # Create a table of results similar to Table 6 in the paper
    models = [
        "YOLO-DP", "YOLOv3", "YOLOv4s", "YOLOv5s", "YOLOv7", "YOLOv8s"
    ]

    pdt_results = {
        "YOLO-DP": {"P": 90.2, "R": 88.0, "mAP@.5": 94.5, "mAP@.5:.95": 67.5, "F1": 0.89, "GFLOPs": 11.7, "FPS": 109},
        "YOLOv3": {"P": 88.5, "R": 88.1, "mAP@.5": 93.4, "mAP@.5:.95": 65.7, "F1": 0.88, "GFLOPs": 155.3, "FPS": 41},
        "YOLOv4s": {"P": 88.8, "R": 88.2, "mAP@.5": 94.7, "mAP@.5:.95": 66.1, "F1": 0.88, "GFLOPs": 20.8, "FPS": 51},
        "YOLOv5s": {"P": 88.9, "R": 88.5, "mAP@.5": 94.2, "mAP@.5:.95": 67.0, "F1": 0.89, "GFLOPs": 16.0, "FPS": 93},
        "YOLOv7": {"P": 87.4, "R": 82.6, "mAP@.5": 90.1, "mAP@.5:.95": 55.5, "F1": 0.85, "GFLOPs": 105.1, "FPS": 32},
        "YOLOv8s": {"P": 88.7, "R": 87.5, "mAP@.5": 94.0, "mAP@.5:.95": 67.9, "F1": 0.88, "GFLOPs": 28.6, "FPS": 60}
    }

    # Print the comparison table
    print("Model Comparison on PDT Dataset:")
    print(f"{'Model':<10} {'Precision':<10} {'Recall':<10} {'mAP@.5':<10} {'mAP@.5:.95':<12} {'F1':<8} {'GFLOPs':<8} {'FPS':<6}")
    print("-" * 80)

    for model in models:
        result = pdt_results[model]
        print(f"{model:<10} {result['P']:<10.1f} {result['R']:<10.1f} {result['mAP@.5']:<10.1f} {result['mAP@.5:.95']:<12.1f} {result['F1']:<8.2f} {result['GFLOPs']:<8.1f} {result['FPS']:<6.0f}")

# Show the performance comparison
compare_models()

## 8. Complete Pipeline Example

In [59]:

def full_pipeline_example():
    """
    Example of running the complete pipeline.
    """
    print("Complete Pipeline Example:")
    print("1. Download PDT dataset")
    dataset_path = download_pdt_dataset()

    print("\n2. Prepare data configuration")
    data_config = prepare_data_config(dataset_path)

    print("\n3. Create YOLO-DP model")
    model = create_yolo_dp_model(pretrained=True)

    print("\n4. Train the model (skipping for this example)")
    # best_weights = train_model(model, data_config, epochs=300, batch_size=16)

    print("\n5. Evaluate the model (skipping for this example)")
    # evaluate_model(best_weights, data_config)

    print("\n6. Run inference (skipping for this example)")
    # visualize_predictions(best_weights, "sample_image.jpg")

    print("\nComplete pipeline example finished!")
    print("In a real implementation, you would:")
    print("1. Download the full PDT dataset from HuggingFace")
    print("2. Implement the YOLO-DP architecture by modifying YOLOv5")
    print("3. Train on a GPU for 300 epochs")
    print("4. Evaluate and compare with other models")
    print("5. Use the model for real-world tree disease detection")

In [60]:
full_pipeline_example()

Complete Pipeline Example:
1. Download PDT dataset
Creating placeholder data for demonstration...
Created placeholder data in /content/datasets/PDT

2. Prepare data configuration
Created data configuration at /content/datasets/PDT/data.yaml

3. Create YOLO-DP model
Creating YOLO-DP model based on YOLOv5s...


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-9 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLO-DP model created (simplified version)!

Note: This is a simplified version of YOLO-DP.
For a full implementation, you would need to modify the YOLOv5 architecture
by integrating the Adaptive Large Scale Selective Kernel and GhostConv modules
as described in the paper.

4. Train the model (skipping for this example)

5. Evaluate the model (skipping for this example)

6. Run inference (skipping for this example)

Complete pipeline example finished!
In a real implementation, you would:
1. Download the full PDT dataset from HuggingFace
2. Implement the YOLO-DP architecture by modifying YOLOv5
3. Train on a GPU for 300 epochs
4. Evaluate and compare with other models
5. Use the model for real-world tree disease detection


## 9. Additional Resources

In [58]:
print("\nAdditional Resources:")
print("1. PDT Dataset: https://huggingface.co/datasets/qwer0213/PDT_dataset")
print("2. Original Paper: https://ar5iv.labs.arxiv.org/html/2409.15679")
print("3. YOLO-DP Code: https://github.com/RuiXing123/PDT_CWC_YOLO-DP")


Additional Resources:
1. PDT Dataset: https://huggingface.co/datasets/qwer0213/PDT_dataset
2. Original Paper: https://ar5iv.labs.arxiv.org/html/2409.15679
3. YOLO-DP Code: https://github.com/RuiXing123/PDT_CWC_YOLO-DP
